# Tensorflow Fundamentals
**In this notebook, we're going to cover some of the most fundamental concepts of tensors using Tensorflow.**

More specifically, we're going to cover:
* [Introduction to tensors](#intro-to-tensor)
    - [`tf.constant()`](#tf-constant)
    - [`tf.Variable()`](#tf-Variable)
    - `tf.range(lower_value, upper_value)`
    - [Create random tensors](#create-random-tensor)
        - `tf.random.normal()`
        - `tf.random.uniform()`
    - [Shuffling the order of tensors](#shuffle-order-of-tensors)
        - `tf.random.shuffle()`
    - [Other ways to create tensors](#otherways-to-create-tensors)
         - `tf.ones()`
         - `tf.zeros()`
         - `tf.constants(numpy_array)`
* [Getting information from tensors](#getting-information-from-tensors)
    - Shape
    - Rank
    - Axis or dimension
    - Size
* [Indexing tensors](#indexing-tensors)
    - Indexing
    - Add extra dimension (`tf.newaxis`, `tf.expand_dims`)
* [Manipulating tensors](#tensor-operation)
    - [Basic operation](#basic-operation)
        - Addition (`tf.math.add()`)
        - Multiplication (`tf.math.multiply()` or `tf.multiply()`)
        - Subtraction
    - [Matrix Multiplication](#matrix-multiplication)
        - `tf.matmul()`
        - Python `@` operator
        - `tf.tensordot()`
        - Different results by doing `tf.transpose()` or `tf.reshape()`
    - [Changing the DataType of a Tensor](#change-datatype)
        - `tf.cast(tensor, dtype=tf.float16)`
    - [Aggregating tensors](#aggregating-tensors)
        - Absolute values, `tf.abs(tensor)`
        - minimum (`tf.reduce_min(tensor)`)
        - maximum (`tf.reduce_max(tensor)`)
        - mean (`tf.reduce_mean(tensor)`)
        - sum of a tensor (`tf.reduce_sum(tensor)`)
        - variance (`tfp.stats.variance(tensor)`  or `tf.math.reduce_variance(tf.cast(tensor, dtype = tf.float32))`)
        - standard deviation (`tf.math.reduce_std(tf.cast(tensor, dtype = tf.float32))`)
        - square (`tf.square(tensor)`)
        - square root (`tf.sqrt(tf.cast(tensor, dtype=tf.float32))`)
        - log (`tf.math.log(tf.cast(tensor, dtype=tf.float32))`)
        - [Find the positional maximum & minimum](#find-position-of-maximum-minimum)
            - maximum positional (`tensor[tf.argmax(tensor)]`)
            - maximum value of tensor (`tf.reduce_max(tensor)`)
            - minimum positional (`tensor[tf.argmin(tensor)]`)
        - [Squeezing a tensor (removing all single dimensions)](#removing-all-single-dimensions)
            - `tf.squeeze(tensor)`
        - [OneHot Encoding](#onehotencoding)
            - `tf.one_hot(some_list, depth = depth)`
* [Tensors & Numpy](#tensorflow-numpy)
* [Using GPUs with Tensorflow (or TPUs)](#using-gpus-with-tensorflow)

## Introduction to Tensors: <a class="anchor" id="intro-to-tensor"></a>

### `tf.constant()` <a class="anchor" id="tf-constant"></a>

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd

In [2]:
print(tf.__version__)  # print Tensorflow version

2.8.0


In [3]:
# Creating tensors with tf.constant()
scalar = tf.constant(7)
scalar

<tf.Tensor: shape=(), dtype=int32, numpy=7>

In [4]:
# check the number of dimensions of a tensor
scalar.ndim

0

In [5]:
# create a vector
vector = tf.constant([10, 10])
vector

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([10, 10])>

In [6]:
# check the dimension of out vector
vector.ndim

1

In [7]:
# create a matrix (has more than 1 dimension)
matrix = tf.constant([[10, 7],
                     [7, 10]])
matrix

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[10,  7],
       [ 7, 10]])>

In [8]:
# check the dimension of our matrix
matrix.ndim

2

In [9]:
# create another matrix
another_matrix = tf.constant([[10., 7.],
                             [3., 2.],
                             [8., 9.]], dtype=tf.float16) # specify the data type with dtype parameter
another_matrix

<tf.Tensor: shape=(3, 2), dtype=float16, numpy=
array([[10.,  7.],
       [ 3.,  2.],
       [ 8.,  9.]], dtype=float16)>

In [10]:
# What's the dimension of the "another_matrix"
another_matrix.ndim

2

In [11]:
# Let's create a tensor
tensor = tf.constant([[[1, 2, 3],
                     [4, 5, 6]],
                     [[7, 8, 9],
                     [10, 11, 12]],
                     [[13, 14, 15],
                     [16, 17, 18]]])
tensor

<tf.Tensor: shape=(3, 2, 3), dtype=int32, numpy=
array([[[ 1,  2,  3],
        [ 4,  5,  6]],

       [[ 7,  8,  9],
        [10, 11, 12]],

       [[13, 14, 15],
        [16, 17, 18]]])>

In [12]:
# check the dimension
tensor.ndim

3

**What we've created so far:**

* **Scalar:** a single number
* **Vector:** a number with direction (e.g., wind speed and direction)
* **Matrix:** a 2-dimension array of numbers
* **Tensor:** an n-dimensaional array of numbers (when n can be any number, a 0-dimensional tensor is a scalar, a 1-dimensional tensor is a vector

### `tf.Variable()` <a class="anchor" id="tf-Variable"></a>

In [13]:
# create the same tensor with tf.Variable() as above
changeable_tensor   = tf.Variable([10, 7])
unchangeable_tensor = tf.constant([10, 7])

changeable_tensor, unchangeable_tensor

(<tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([10,  7])>,
 <tf.Tensor: shape=(2,), dtype=int32, numpy=array([10,  7])>)

In [14]:
# Let's try change one of the elements in our changeable tensor
changeable_tensor[0] = 7
changeable_tensor

TypeError: 'ResourceVariable' object does not support item assignment

In [15]:
# How about we try .assign()
changeable_tensor[0].assign(7)
changeable_tensor

<tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([7, 7])>

In [16]:
# Let's try change our unchangable tensor
unchangeable_tensor[0] = 7

TypeError: 'tensorflow.python.framework.ops.EagerTensor' object does not support item assignment

In [17]:
unchangeable_tensor[0].assign(7)

AttributeError: 'tensorflow.python.framework.ops.EagerTensor' object has no attribute 'assign'

**🔑Note:** Rarely in practice will you need to decide whether to use `tf.constant` or `tf.Variable` to create tensors, as Tensorflow does this for you. However, if in doubt, use `tf.constant` and change it later if needed.

### Creating random tensors <a class="anchor" id="create-random-tensor"></a>

Random tensors are tensors of some abitary size which contain random numbers.

In [22]:
# A uniform distribution, sometimes also known as a rectangular distribution, is a distribution that has constant probability.

# create two random (but the same) tensors
random_1 = tf.random.Generator.from_seed(42)  # set seed for reproducibility
random_1 = random_1.normal(shape = (3, 2))
random_2 = tf.random.Generator.from_seed(42)
random_2 = random_2.normal(shape = (3, 2))

# Are they equal?
random_1, random_2, random_1 == random_2

(<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[-0.7565803 , -0.06854702],
        [ 0.07595026, -1.2573844 ],
        [-0.23193765, -1.8107855 ]], dtype=float32)>,
 <tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[-0.7565803 , -0.06854702],
        [ 0.07595026, -1.2573844 ],
        [-0.23193765, -1.8107855 ]], dtype=float32)>,
 <tf.Tensor: shape=(3, 2), dtype=bool, numpy=
 array([[ True,  True],
        [ True,  True],
        [ True,  True]])>)

### Shuffling the order of tensors <a class="anchor" id="shuffle-order-of-tensors"></a>

In [30]:
# shuffle a tensor (valuable for when you want to shuffle your data so the inherent order doesn't effect learning)
not_shuffled = tf.constant([[10, 7],
                           [3, 4],
                           [2, 5]])
# shuffle our non_shuffled
tf.random.shuffle(not_shuffled, seed=42)  # The tensor is shuffled along dimension 0

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[ 2,  5],
       [ 3,  4],
       [10,  7]])>

**🛠 Read through Tensorflow documentation on random seed generation: https://www.tensorflow.org/api_docs/python/tf/random/set_seed and practice writing 5 random tensors and shuffle them.** 

It looks like if we want out shuffled tensors to be in the same order, we've got to use the global level random seed as well as the operation level random seed:
> Rule 4: "If both the global and the operation seed are set: Both seeds are used in conjunction to determine the random sequence."

In [32]:
tf.random.set_seed(42)                    # global level random seed
tf.random.shuffle(not_shuffled, seed=42)  # operation level random seed

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4],
       [ 2,  5]])>

### Other ways to create tensors: <a id="otherways-to-create-tensors"><a/>

In [33]:
# create a tensor of all ones
tf.ones([10, 7])

<tf.Tensor: shape=(10, 7), dtype=float32, numpy=
array([[1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.]], dtype=float32)>

In [35]:
# create tensors of all zeros
tf.zeros([3, 4], dtype=tf.int32)

<tf.Tensor: shape=(3, 4), dtype=int32, numpy=
array([[0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0]])>

In [36]:
# you can also turn Numpy arrays into tensors
# The main difference between Numpy arrays and Tensorflow tensors is that tensors 
# can be run on a GPU (much faster for numerical computation)
numpy_A = np.arange(1, 25, dtype=np.int32)  # create a Numpy array between 1 and 25
numpy_A

# X = tf.constant(some_matrix)  # capital for matrix or tensor
# y = tf.constant(vector)       # non-capital for vector

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24])

In [38]:
A = tf.constant(numpy_A, shape = (2, 3, 4))
A

<tf.Tensor: shape=(2, 3, 4), dtype=int32, numpy=
array([[[ 1,  2,  3,  4],
        [ 5,  6,  7,  8],
        [ 9, 10, 11, 12]],

       [[13, 14, 15, 16],
        [17, 18, 19, 20],
        [21, 22, 23, 24]]])>

## Getting Information from tensors <a id="getting-information-from-tensors"></a>

**When dealing with tensors you probably want to be aware of the following attributes:**
- Shape
- Rank
- Axis or dimension
- Size

**Some Tensor attributes:**

| Attribute | Meaning | Code |
| :--: | :-- | :--: |
| Shape | The length (number of elements) of each of the dimensions of a tensor. | `tensor.shape` |
| Rank | The number of tensor dimensions. A scalar has rank 0, a vector has rank 1, a matrix is rank 2, a tensor has rank n. | `tensor.ndim` |
| Axis or dimension | A particular dimension of a tensor | `tensor[0]`, `tensor[:, 1]`, ... |
| Size | The total number of items in the tensor. | `tf.size(tensor)` |

In [2]:
# create a rank 4 tensor (4 dimensions)
rank_4_tensor = tf.zeros(shape = [2, 3, 4, 5])
rank_4_tensor

<tf.Tensor: shape=(2, 3, 4, 5), dtype=float32, numpy=
array([[[[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]]],


       [[[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]]]], dtype=float32)>

In [3]:
rank_4_tensor[0]

<tf.Tensor: shape=(3, 4, 5), dtype=float32, numpy=
array([[[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]]], dtype=float32)>

In [4]:
rank_4_tensor.shape, rank_4_tensor.ndim, tf.size(rank_4_tensor)

# 2 * 3 * 4 * 5 = 120

(TensorShape([2, 3, 4, 5]), 4, <tf.Tensor: shape=(), dtype=int32, numpy=120>)

In [5]:
# get various attributes of our tensors
print("Datatype of every element:", rank_4_tensor.dtype)
print("Number of dimensions (rank):", rank_4_tensor.ndim)
print("Shape of tensor:", rank_4_tensor.shape)
print("Elements along the 0 axis:", rank_4_tensor.shape[0])
print("Elements along the last axis:", rank_4_tensor.shape[-1])
print("Total number of elements in our tensor:", tf.size(rank_4_tensor))
print("Total number of elements in our tensor:", tf.size(rank_4_tensor).numpy())

Datatype of every element: <dtype: 'float32'>
Number of dimensions (rank): 4
Shape of tensor: (2, 3, 4, 5)
Elements along the 0 axis: 2
Elements along the last axis: 5
Total number of elements in our tensor: tf.Tensor(120, shape=(), dtype=int32)
Total number of elements in our tensor: 120


## Indexing tensors <a id="indexing-tensors"></a>

Tensors can be indexed just like Python lists.

In [6]:
# Get the first 2 elements of each dimension
rank_4_tensor[:2, :2, :2, :2]

<tf.Tensor: shape=(2, 2, 2, 2), dtype=float32, numpy=
array([[[[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]]],


       [[[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]]]], dtype=float32)>

In [7]:
# Get the first element from  each dimension from each index except for the final one
rank_4_tensor[:1, :1, :1, :]

<tf.Tensor: shape=(1, 1, 1, 5), dtype=float32, numpy=array([[[[0., 0., 0., 0., 0.]]]], dtype=float32)>

In [8]:
rank_4_tensor[:1, :1, :, :1]

<tf.Tensor: shape=(1, 1, 4, 1), dtype=float32, numpy=
array([[[[0.],
         [0.],
         [0.],
         [0.]]]], dtype=float32)>

In [9]:
# create a rank 2 tensor (2 dimensions)
rank_2_tensor = tf.constant([[10, 7],
                            [3, 4]])
rank_2_tensor.shape, rank_2_tensor.ndim

(TensorShape([2, 2]), 2)

In [10]:
# get the last item of each of row of our rank_2_tensor
rank_2_tensor[:, -1]

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([7, 4])>

In [11]:
# Add in extra dimension to our rank_2_tensor
rank_3_tensor = rank_2_tensor[..., tf.newaxis]
rank_3_tensor

<tf.Tensor: shape=(2, 2, 1), dtype=int32, numpy=
array([[[10],
        [ 7]],

       [[ 3],
        [ 4]]])>

In [12]:
# Alternative to tf.newaxis
tf.expand_dims(rank_2_tensor, axis=-1)  # "-1" means expand the final axis

<tf.Tensor: shape=(2, 2, 1), dtype=int32, numpy=
array([[[10],
        [ 7]],

       [[ 3],
        [ 4]]])>

In [13]:
tf.expand_dims(rank_2_tensor, axis=0)  # expand the 0-axis

<tf.Tensor: shape=(1, 2, 2), dtype=int32, numpy=
array([[[10,  7],
        [ 3,  4]]])>

In [14]:
tf.expand_dims(rank_2_tensor, axis=1)  # expand the 1-axis

<tf.Tensor: shape=(2, 1, 2), dtype=int32, numpy=
array([[[10,  7]],

       [[ 3,  4]]])>

## Manipulating tensors (Tensor Operation) <a id="tensor-operation"></a>

### Basic operations <a id="basic-operation"></a>

In [15]:
# You can add values to a tensor using the addition operator
tensor = tf.constant([[10, 7], [3, 4]])
tensor + 10

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[20, 17],
       [13, 14]])>

In [16]:
# Multiplication also works
tensor * 10

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[100,  70],
       [ 30,  40]])>

In [17]:
# Subtraction if you want
tensor - 10

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[ 0, -3],
       [-7, -6]])>

In [18]:
# we can use the Tensorflow built-in function too
tf.multiply(tensor, 10)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[100,  70],
       [ 30,  40]])>

### Matrix Multiplication <a id="matrix-multiplication"></a>

In machine learning, matrix multiplication is one of the most common tensor operations.

In [19]:
# matrix multiplication in Tensorflow
print(f"{tensor}\n")
tf.matmul(tensor, tensor)

[[10  7]
 [ 3  4]]



<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[121,  98],
       [ 42,  37]])>

In [20]:
tensor * tensor

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[100,  49],
       [  9,  16]])>

In [22]:
# matrix multiplication with Python operator "@"
tensor @ tensor

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[121,  98],
       [ 42,  37]])>

In [23]:
# create a tensor (3, 2).
X = tf.constant([[1, 2],
                [3, 4],
                [5, 6]])
# create another (3, 2) tensor
Y = tf.constant([[7, 8],
                [9, 10],
                [11, 12]])
X, Y

(<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[1, 2],
        [3, 4],
        [5, 6]])>,
 <tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[ 7,  8],
        [ 9, 10],
        [11, 12]])>)

In [24]:
# try to matrix multiply tensors of same shape
X @ Y

InvalidArgumentError: Matrix size-incompatible: In[0]: [3,2], In[1]: [3,2] [Op:MatMul]

📖 **Resource:** Info and example of matrix multiplication - https://www.mathsisfun.com/algebra/matrix-multiplying.html

📏 **There are two rules our tensors (or matrices) need to fulfill if we're going to matrix multiple them:**
1. The inner dimensions must match 
2. The resulting matrix has the shape of the outer dimensions.

In [25]:
# let's change the shape of Y
tf.reshape(Y, shape = (2, 3))

<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
array([[ 7,  8,  9],
       [10, 11, 12]])>

In [26]:
# try to multiply X by reshaped Y
X @ tf.reshape(Y, shape = (2, 3))

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 27,  30,  33],
       [ 61,  68,  75],
       [ 95, 106, 117]])>

In [27]:
tf.matmul(X, tf.reshape(Y, shape = (2, 3)))

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 27,  30,  33],
       [ 61,  68,  75],
       [ 95, 106, 117]])>

In [28]:
# try change the shape of X instead of Y
tf.matmul(tf.reshape(X, shape=(2, 3)), Y)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[ 58,  64],
       [139, 154]])>

**The dot product**

Matrix multiplication is also known referred to as the dot product.

You can perform matrix multiplication using:
- `tf.matmul()`
- `tf.tesnordot()`
- Python `@` operator

Generally, when performing matrix multiplication on two tensors and one of the axes doesn't line up, you will transpose (rather than reshape) one of the tensors to get statisfy the matrix multiplication rules.

In [29]:
# perform the dot product on X and Y (requires X or Y to be transposed)
tf.tensordot(tf.transpose(X), Y, axes=1)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[ 89,  98],
       [116, 128]])>

In [30]:
# perform matrix multiplication between X and Y (transposed)
tf.matmul(X, tf.transpose(Y))

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 23,  29,  35],
       [ 53,  67,  81],
       [ 83, 105, 127]])>

In [31]:
# perform matrix multiplication between X and Y (reshaped)
tf.matmul(X, tf.reshape(Y, shape = (2, 3)))

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 27,  30,  33],
       [ 61,  68,  75],
       [ 95, 106, 117]])>

In [32]:
# check the values of Y, reshape Y and transposed Y
print("Normal Y:")
print(Y, "\n")

print("Y reshaped to (2, 3):")
print(tf.reshape(Y, shape = (2, 3)), "\n")

print("Y transposed:")
print(tf.transpose(Y))

Normal Y:
tf.Tensor(
[[ 7  8]
 [ 9 10]
 [11 12]], shape=(3, 2), dtype=int32) 

Y reshaped to (2, 3):
tf.Tensor(
[[ 7  8  9]
 [10 11 12]], shape=(2, 3), dtype=int32) 

Y transposed:
tf.Tensor(
[[ 7  9 11]
 [ 8 10 12]], shape=(2, 3), dtype=int32)


### Changing the DataType of a Tensor <a id="change-datatype"></a> 

In [37]:
# create a new tensor with default datatype (float32)
B = tf.constant([1.7, 7.4])
B, B.dtype

(<tf.Tensor: shape=(2,), dtype=float32, numpy=array([1.7, 7.4], dtype=float32)>,
 tf.float32)

In [38]:
C = tf.constant([7, 10])
C, C.dtype

(<tf.Tensor: shape=(2,), dtype=int32, numpy=array([ 7, 10])>, tf.int32)

In [39]:
# change from float32 to float16 (reduced precision)
D = tf.cast(B, dtype=tf.float16)
D, D.dtype

(<tf.Tensor: shape=(2,), dtype=float16, numpy=array([1.7, 7.4], dtype=float16)>,
 tf.float16)

In [40]:
E = tf.cast(C, dtype = tf.float32)
E

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([ 7., 10.], dtype=float32)>

In [41]:
E_float16 = tf.cast(E, dtype=tf.float16)
E_float16

<tf.Tensor: shape=(2,), dtype=float16, numpy=array([ 7., 10.], dtype=float16)>

## Aggregating Tensors <a id="aggregating-tensors"></a>

Aggregating tensors = condensing them from multiple values down to a smaller amount of values.

In [3]:
# get the absolute values
D = tf.constant([-7, -10])

tf.abs(D)

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([ 7, 10])>

**let's go through the following forms of aggregation:**
- Get the minimum
- Get the maximum
- Get the mean of a tensor
- Get the sum of a tensor

In [4]:
# create a random tensor with values between 0 and 100 of size 50
E = tf.constant(np.random.randint(0, 100, size=50))
E

<tf.Tensor: shape=(50,), dtype=int32, numpy=
array([37,  5, 48, 33, 44, 61, 21, 85, 57, 25, 90, 41, 25, 26, 55, 80, 73,
       12, 97, 72, 82, 83, 21, 10, 79, 58, 52, 63, 64, 50, 75, 52, 48, 79,
       90, 85, 92, 81, 69, 80, 99,  9, 53, 38, 92, 90, 90, 29, 38, 95])>

In [5]:
tf.size(E), E.shape, E.ndim

(<tf.Tensor: shape=(), dtype=int32, numpy=50>, TensorShape([50]), 1)

In [6]:
# find the minimum
tf.reduce_min(E)

<tf.Tensor: shape=(), dtype=int32, numpy=5>

In [7]:
# find the maximum
tf.reduce_max(E)

<tf.Tensor: shape=(), dtype=int32, numpy=99>

In [8]:
# find the mean
tf.reduce_mean(E)

<tf.Tensor: shape=(), dtype=int32, numpy=58>

In [9]:
# find the sum
tf.reduce_sum(E)

<tf.Tensor: shape=(), dtype=int32, numpy=2933>

🛠 **Exercise:** With what we've just learned, find the variance and standard deviation of our `E` tensor using Tensorflow methods.

In [10]:
# find the variance of our tensor
import tensorflow_probability as tfp

tfp.stats.variance(E)

<tf.Tensor: shape=(), dtype=int32, numpy=721>

In [15]:
# find the variance of our E tensor
tf.math.reduce_variance(tf.cast(E, dtype = tf.float32))

<tf.Tensor: shape=(), dtype=float32, numpy=720.58435>

In [12]:
tf.math.reduce_std(E)

TypeError: Input must be either real or complex. Received integer type <dtype: 'int32'>.

In [13]:
# find the standard deviation
tf.math.reduce_std(tf.cast(E, dtype = tf.float32))

<tf.Tensor: shape=(), dtype=float32, numpy=26.843702>

### Find the positional maximum & minimum <a id="find-position-of-maximum-minimum"></a>



In [17]:
# create a new tensor for finding positional minimum and maximum
tf.random.set_seed(42)
F = tf.random.uniform(shape = [50])
F

<tf.Tensor: shape=(50,), dtype=float32, numpy=
array([0.6645621 , 0.44100678, 0.3528825 , 0.46448255, 0.03366041,
       0.68467236, 0.74011743, 0.8724445 , 0.22632635, 0.22319686,
       0.3103881 , 0.7223358 , 0.13318717, 0.5480639 , 0.5746088 ,
       0.8996835 , 0.00946367, 0.5212307 , 0.6345445 , 0.1993283 ,
       0.72942245, 0.54583454, 0.10756552, 0.6767061 , 0.6602763 ,
       0.33695042, 0.60141766, 0.21062577, 0.8527372 , 0.44062173,
       0.9485276 , 0.23752594, 0.81179297, 0.5263394 , 0.494308  ,
       0.21612847, 0.8457197 , 0.8718841 , 0.3083862 , 0.6868038 ,
       0.23764038, 0.7817228 , 0.9671384 , 0.06870162, 0.79873943,
       0.66028714, 0.5871513 , 0.16461694, 0.7381023 , 0.32054043],
      dtype=float32)>

In [18]:
# find the positional maximum
tf.argmax(F)

<tf.Tensor: shape=(), dtype=int64, numpy=42>

In [19]:
# index on our larges value position
F[tf.argmax(F)]

<tf.Tensor: shape=(), dtype=float32, numpy=0.9671384>

In [20]:
# find the max value of F
tf.reduce_max(F)

<tf.Tensor: shape=(), dtype=float32, numpy=0.9671384>

In [21]:
# check for equality
assert F[tf.argmax(F)] == tf.reduce_max(F)

In [22]:
# find the positional minimum
tf.argmin(F)

<tf.Tensor: shape=(), dtype=int64, numpy=16>

In [23]:
# find the minimum using the positional minimum index
F[tf.argmin(F)]

<tf.Tensor: shape=(), dtype=float32, numpy=0.009463668>

### Squeezing a tensor (removing all single dimensions) <a id="removing-all-single-dimensions"></a>

In [25]:
# create a tensor to get started
tf.random.set_seed(42)
G = tf.constant(tf.random.uniform(shape = [50]), shape = (1, 1, 1, 1, 50))
G

<tf.Tensor: shape=(1, 1, 1, 1, 50), dtype=float32, numpy=
array([[[[[0.6645621 , 0.44100678, 0.3528825 , 0.46448255, 0.03366041,
           0.68467236, 0.74011743, 0.8724445 , 0.22632635, 0.22319686,
           0.3103881 , 0.7223358 , 0.13318717, 0.5480639 , 0.5746088 ,
           0.8996835 , 0.00946367, 0.5212307 , 0.6345445 , 0.1993283 ,
           0.72942245, 0.54583454, 0.10756552, 0.6767061 , 0.6602763 ,
           0.33695042, 0.60141766, 0.21062577, 0.8527372 , 0.44062173,
           0.9485276 , 0.23752594, 0.81179297, 0.5263394 , 0.494308  ,
           0.21612847, 0.8457197 , 0.8718841 , 0.3083862 , 0.6868038 ,
           0.23764038, 0.7817228 , 0.9671384 , 0.06870162, 0.79873943,
           0.66028714, 0.5871513 , 0.16461694, 0.7381023 , 0.32054043]]]]],
      dtype=float32)>

In [26]:
G.shape

TensorShape([1, 1, 1, 1, 50])

In [27]:
G_squeezed = tf.squeeze(G)
G_squeezed, G_squeezed.shape

(<tf.Tensor: shape=(50,), dtype=float32, numpy=
 array([0.6645621 , 0.44100678, 0.3528825 , 0.46448255, 0.03366041,
        0.68467236, 0.74011743, 0.8724445 , 0.22632635, 0.22319686,
        0.3103881 , 0.7223358 , 0.13318717, 0.5480639 , 0.5746088 ,
        0.8996835 , 0.00946367, 0.5212307 , 0.6345445 , 0.1993283 ,
        0.72942245, 0.54583454, 0.10756552, 0.6767061 , 0.6602763 ,
        0.33695042, 0.60141766, 0.21062577, 0.8527372 , 0.44062173,
        0.9485276 , 0.23752594, 0.81179297, 0.5263394 , 0.494308  ,
        0.21612847, 0.8457197 , 0.8718841 , 0.3083862 , 0.6868038 ,
        0.23764038, 0.7817228 , 0.9671384 , 0.06870162, 0.79873943,
        0.66028714, 0.5871513 , 0.16461694, 0.7381023 , 0.32054043],
       dtype=float32)>,
 TensorShape([50]))

### OneHot Encoding Tensors <a id="onehotencoding"></a>

In [28]:
# create a list of indices
some_list = [0, 1, 2, 3]  # could be red, green, blue, purple

# one hot encode our list of indices
tf.one_hot(some_list, depth = 4)

<tf.Tensor: shape=(4, 4), dtype=float32, numpy=
array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]], dtype=float32)>

In [31]:
tf.one_hot(some_list, depth = 3)

<tf.Tensor: shape=(4, 3), dtype=float32, numpy=
array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 0.]], dtype=float32)>

In [32]:
tf.one_hot(some_list, depth = 5)

<tf.Tensor: shape=(4, 5), dtype=float32, numpy=
array([[1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.]], dtype=float32)>

In [36]:
# specify custom values for one hot encoding
tf.one_hot(some_list, depth=4, on_value=+1, off_value=-1)

<tf.Tensor: shape=(4, 4), dtype=int32, numpy=
array([[ 1, -1, -1, -1],
       [-1,  1, -1, -1],
       [-1, -1,  1, -1],
       [-1, -1, -1,  1]])>

### Squaring, log, square root

In [38]:
# create a new tensor
H = tf.range(1, 10)
H

<tf.Tensor: shape=(9,), dtype=int32, numpy=array([1, 2, 3, 4, 5, 6, 7, 8, 9])>

In [39]:
# square it
tf.square(H)

<tf.Tensor: shape=(9,), dtype=int32, numpy=array([ 1,  4,  9, 16, 25, 36, 49, 64, 81])>

In [40]:
# find the square root
tf.sqrt(tf.cast(H, dtype=tf.float32))

<tf.Tensor: shape=(9,), dtype=float32, numpy=
array([1.       , 1.4142135, 1.7320508, 2.       , 2.2360678, 2.4494896,
       2.6457512, 2.828427 , 3.       ], dtype=float32)>

In [41]:
# find the log
tf.math.log(tf.cast(H, dtype=tf.float32))

<tf.Tensor: shape=(9,), dtype=float32, numpy=
array([0.       , 0.6931472, 1.0986123, 1.3862944, 1.609438 , 1.7917595,
       1.9459102, 2.0794415, 2.1972246], dtype=float32)>

## TensorFlow & NumPy <a id="tensorflow-numpy"></a>

TensorFlow interacts beautifully with NumPy arrays.

🔑 **Note:** One of the main fifferences between a TensorFlow tensor and a NumPy array is that a TensorFlow tensor can be run on a GPU or TPU (for faster numerical processing).

In [42]:
# create a tensor directly from a NumPy array
J = tf.constant(np.array([3., 7., 10.]))
J

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([ 3.,  7., 10.])>

In [43]:
# convert our tensor back to a NumPy array
np.array(J), type(np.array(J))

(array([ 3.,  7., 10.]), numpy.ndarray)

In [44]:
# convert tensor J to a NumPy array
J.numpy(), type(J.numpy())

(array([ 3.,  7., 10.]), numpy.ndarray)

In [46]:
J = tf.constant([3.])
J.numpy()[0]

3.0

In [47]:
# the default types of each are slightly different
numpy_J  = tf.constant(np.array([3., 7., 10.]))
tensor_J = tf.constant([3., 7., 10.])

# check the datatypes of each
numpy_J.dtype, tensor_J.dtype

(tf.float64, tf.float32)

## Using GPUs with Tensorflow <a id="using-gpus-with-tensorflow"><a/>

In [48]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [49]:
tf.config.list_physical_devices("GPU")

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [50]:
!nvidia-smi

Sat May 14 20:28:58 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 512.59       Driver Version: 512.59       CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   50C    P8     4W /  N/A |   4346MiB /  6144MiB |      1%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

🔑 **Note:** If you have access to a CUDA-enabled GPU, TensorFlow will automatically use it whenver possible.